# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### S&P 500 

In [2]:
ltdate = int(time.time())

url = ('https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?'+
       f'period1=599616000&period2={ltdate}&interval=1d&events=history')

df = pd.read_csv(url).set_index('Date')

close = df['Adj Close']
close.index = pd.to_datetime(close.index)
lt = pd.Series({close.index[-1]: close[-1]})
final = close.resample('MS').mean().append(lt)
final.name = 'GSPC'
final.to_csv(data_dir / 'sp500.csv', index_label='date') 

data = pd.DataFrame()
data['GSPC'] = close.loc['2017':]
data['MA'] = close.rolling(253).mean().loc['2017':]
data.to_csv(data_dir / 'sp500_recent.csv', index_label='date')

ltdate = dtxt(close.index[-1])['day1']
ltval = close.iloc[-1]
hival = df.High.iloc[-253:].max()
chhi = (1 - ltval/hival) * 100
hidt = dtxt(df.High.iloc[-253:].idxmax())['day1']
loval = df.Low.iloc[-253:].min()
chlo = abs((1 - ltval/loval) * 100)
lodt = dtxt(df.Low.iloc[-253:].idxmin())['day1']
avval = df['Adj Close'].iloc[-253:].mean()
chav = (1 - ltval/avval) * 100
chtxt = value_text(-chav, style='above_below', threshold=0.1)
cline = '(see {\color{black!18!white}\\textbf{---}}'
text = (f'The S\&P 500 closed at {ltval:,.0f} on {ltdate}. The '+
        f'index is currently {chhi:.1f} percent below its one-year '+
        f'high of {hival:,.0f} on {hidt}, and {chlo:.1f} percent '+
        f'above its one-year low of {loval:,.0f} on {lodt}. The '+
        f'average over the past year is {avval:,.0f}; the index is '+
        f'{chtxt} its one-year moving average {cline}).')
write_txt(text_dir / 'sp500.txt', text)
print(text)

The S\&P 500 closed at 4,501 on February 4, 2022. The index is currently 6.6 percent below its one-year high of 4,819 on January 4, 2022, and 20.9 percent above its one-year low of 3,723 on March 4, 2021. The average over the past year is 4,344; the index is 3.6 percent above its one-year moving average (see {\color{black!18!white}\textbf{---}}).


### Real Interest Rates

In [3]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=df361c47287a0589560a46dad7d610cb&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

n = {'RIFLGFCY10_XII_N.B': 'Ten-year',
     'RIFLGFCY05_XII_N.B': 'Five-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'real_rates.csv', index_label='date', float_format='%g')

In [4]:
write_txt(text_dir / 'real_10y_node.txt', 
          end_node(data['Ten-year'], 'red', digits=2))
write_txt(text_dir / 'real_5y_node.txt', 
          end_node(data['Five-year'], 'violet', digits=2))

ltdate = dtxt(df.index[-1])['day1']
prdate = dtxt(df.index[-22])['day1']
ltval = df['Ten-year'].iloc[-1]
ltval5 = df['Five-year'].iloc[-1]
prval = df['Ten-year'].iloc[-22]
prval5 = df['Five-year'].iloc[-22]
cline = '(see {\color{red}\\textbf{---}})'
cline2 = '(see {\color{violet}\\textbf{---}})'
text = ('US Treasury inflation-indexed securities are used to measure '+
        'real interest rates. Treasury yields in general are a good '+
        'proxy for low-risk rates, and the treasury offers specific '+
        'securities that have interest payments indexed to the consumer '+
        'price index (CPI) rate of inflation. \n\n The real yield for '+
        f'such treasuries with ten years to maturity is {ltval:.2f} percent, '+
        f'as of {ltdate} {cline}, compared to {prval:.2f} percent on '+
        f'{prdate}. For a five-year maturity, the real yield is {ltval5:.2f} '+
        f'percent in the latest data {cline2}, compared to {prval5:.2f} '+
        'percent a month prior. ')
write_txt(text_dir / 'real_rates_basic.txt', text)
print(text)

US Treasury inflation-indexed securities are used to measure real interest rates. Treasury yields in general are a good proxy for low-risk rates, and the treasury offers specific securities that have interest payments indexed to the consumer price index (CPI) rate of inflation. 

 The real yield for such treasuries with ten years to maturity is -0.56 percent, as of February 3, 2022 (see {\color{red}\textbf{---}}), compared to -0.91 percent on January 4, 2022. For a five-year maturity, the real yield is -1.10 percent in the latest data (see {\color{violet}\textbf{---}}), compared to -1.56 percent a month prior. 


### Interest rates

In [5]:
# Taylor Rule suggested Fed Funds rate
p = (pd.read_csv(data_dir / 'pce_pi.csv', parse_dates=['date'])
       .set_index('date')['CORE']).resample('QS').mean() * 0.5

y = ((nipa_df(retrieve_table('T10106')['Data'], ['A191RX']))
      .loc['1989':, 'A191RX'])

y_p = fred_df('GDPPOT')['VALUE'] * 1_000

o = (y - (y_p)).divide(y_p).dropna()

taylor_ff = (p + 3 + (0.5*(p - 2)) + (1*(o*100)))

taylor_ff.name = 'Value'

taylor_ff.dropna().to_csv(data_dir / 'taylor.csv', index_label='date')

In [6]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=a2738d013c65f34c46f905d1d7913f78&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

n = {'RIFLGFCY10_N.B': 'Ten-year',
    'RIFLGFCY30_N.B': 'Thirty-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY05_N.B': 'Five-year',
    'RIFLGFCY02_N.B': 'Two-year',
    'RIFLGFCM01_N.B': 'One-month',
    'RIFLGFCY01_N.B': 'One-year',
    'RIFLGFCY20_N.B': 'Twenty-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

ldate = dtxt(data.index[-1])['day1']
tenlt = df['Ten-year'].iloc[-1]
tenpr = df['Ten-year'].iloc[-252]
twolt = df['Two-year'].iloc[-1]
twopr = df['Two-year'].iloc[-252]
val89 = df['Ten-year'].loc['1989'].mean()
lowval = data['Ten-year'].min()
lowmon = dtxt(data['Ten-year'].idxmin())['mon1']
cline10 = '(see {\color{red!90}\\textbf{---}})'
cline2 = '(see {\color{blue}\\textbf{---}})'
ch2yr = value_text(df['Two-year'].diff(252).iloc[-1], 
                   'increase_by', ptype='pp')
ff = fred_df('FEDTARMD')['VALUE']
ffd = ff.diff(2).iloc[-1]
fv = 'raise' if ffd > 0.3 else 'lower' if ffd <0.3 else 'maintain'
text = ('United States Treasury securities, or treasuries, are the '+
        'asset created by federal government borrowing. The treasuries '+
        'market is traditionally considered both very low risk and fairly '+
        'liquid. The yield on these securities has fallen over time, '+
        'from an average 10-year treasury bond annual yield of '+
        f'{val89:.1f} percent in 1989 to an average of {lowval:.2f} '+
        f'percent in {lowmon}. As of {ldate}, the constant maturity yield '+
        f'for 10-year treasury bonds is {tenlt} percent {cline10}, '+
        f'compared to {tenpr:.2f} percent one year prior. \n\nYields '+
        'have also fallen since 1989 for shorter-duration treasuries, '+
        f'though shorter-duration treasuries are more acutely affected '+
        'by changes in the key interest rate set by the Federal Reserve. '+
        f'Over the past year, yields on two-year treasuries have {ch2yr}, '+
        f'as the Federal Reserve is expected to {fv} interest rates. '+
        f'As of {ldate}, the annual yield on two-year treasuries is '+
        f'{twolt} percent {cline}. ')
write_txt(text_dir / 'rates_basic.txt', text)
print(text, '\n')
cline = '(see {\color{blue!60!black}\\textbf{---}})'
val = f'{data["Fed Funds"].iloc[-1]:.1f} percent'
text = (f'The effective fed funds rate is {val}, as of {ldate} {cline}.')
write_txt(text_dir / 'rates_ff.txt', text)
print(text)

rows = ['Three-month', 'Two-year', 'Five-year', 'Ten-year', 'Thirty-year']
columns = [-1, -2, -5, -22, -64, -127, -252, -1251]
data2 = df[rows].iloc[columns].T
data2.columns = [dtxt(i)['day2'] for i in pd.to_datetime(data2.keys())]

(data2.applymap('{:,.2f}'.format)
      .to_csv(data_dir / 'treasury_rates.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

#data2

United States Treasury securities, or treasuries, are the asset created by federal government borrowing. The treasuries market is traditionally considered both very low risk and fairly liquid. The yield on these securities has fallen over time, from an average 10-year treasury bond annual yield of 8.5 percent in 1989 to an average of 0.62 percent in July 2020. As of February 3, 2022, the constant maturity yield for 10-year treasury bonds is 1.82 percent (see {\color{red!90}\textbf{---}}), compared to 1.15 percent one year prior. 

Yields have also fallen since 1989 for shorter-duration treasuries, though shorter-duration treasuries are more acutely affected by changes in the key interest rate set by the Federal Reserve. Over the past year, yields on two-year treasuries have increased by 1.1 percentage points, as the Federal Reserve is expected to raise interest rates. As of February 3, 2022, the annual yield on two-year treasuries is 1.19 percent (see {\color{red}\textbf{---}}).  

The

### Yield Curve

In [7]:
i = {'One-month': 1, 'Three-month': 2, 'One-year': 3, 'Two-year': 4, 
     'Five-year': 5, 'Ten-year': 6, 'Twenty-year': 7, 'Thirty-year': 8}
tbl = pd.DataFrame()
for v, c in [(-1, 'value'), (-252, 'oneyear'), (-252*5, 'fiveyear')]:
    col = df[i.keys()].iloc[v]
    col.index = col.index.map(i)
    tbl[c] = col
tbl.index.name = 'number'
tbl['alignment'] = 270

tbl.to_csv(data_dir / 'yc.csv', float_format='%g')

### Spread

In [8]:
data = df[df['Ten-year'] != 'ND'].astype('float')
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2012':].to_csv(data_dir / 'spread.csv', index_label='date', 
                           float_format='%g', header=True)

In [9]:
data = df[df['Ten-year'] != 'ND'].astype('float')
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2012':].to_csv(data_dir / 'spread.csv', index_label='date', 
                           float_format='%g', header=True)

node = end_node(spread['Ten-3M'], 'blue', digits=2)
write_txt(text_dir / 'spread_node.txt', node)

node = end_node(spread['Ten-2Y'], 'red!90!black', digits=2)
write_txt(text_dir / 'spread_node2.txt', node)

def pp(value):
    return 'percentage points' if value > 1.01 else 'percentage point'

lt3 = spread['Ten-3M'].iloc[-1]
lt3t = f'{lt3:.2f} {pp(lt3)}'
pr3 = spread['Ten-3M'].iloc[-252]
pr3t = f'{pr3:.2f} {pp(pr3)}'
lt2 = spread['Ten-2Y'].iloc[-1]
lt2t = f'{lt2:.2f} {pp(lt2)}'
pr2 = spread['Ten-2Y'].iloc[-252]
pr2t = f'{pr2:.2f} {pp(pr2)}'

text = (f'As of {ldate}, the spread between a 10-year treasury bond and '+
        f'a three-month treasury bill is {lt3t} '+
        '(see {\color{blue}\\textbf{---}}), compared to '+
        f'{pr3t} one year prior. The spread between 10-year '+
        'and 2-year treasuries (see {\color{red!90!black}\\textbf{---}}) '+
        f'is {lt2t} on {ldate}, and '+
        f'{pr2t} one year prior.')

write_txt(text_dir / 'spread_basic.txt', text)
print(text)

As of February 3, 2022, the spread between a 10-year treasury bond and a three-month treasury bill is 1.62 percentage points (see {\color{blue}\textbf{---}}), compared to 1.11 percentage points one year prior. The spread between 10-year and 2-year treasuries (see {\color{red!90!black}\textbf{---}}) is 0.63 percentage point on February 3, 2022, and 1.04 percentage points one year prior.


### Gold Price

In [10]:
# df = fred_df('GOLDAMGBD228NLBM').dropna()
# data = (df.resample('M').mean().iloc[:-1].append(df.iloc[-1]))
# data.to_csv(data_dir / 'gold.csv', index_label='date')

# ltdate = dtxt(data.index[-1])['day1']
# ltval = data.VALUE.iloc[-1]
# prdate = dtxt(data.index[-13])['mon1']
# prval = data.VALUE.iloc[-13]
# grdate = dtxt(data.loc['2006':'2011', 'VALUE'].idxmax())['mon1']
# grval = data.loc['2006':'2011', 'VALUE'].max()

# text = (f'As of {ltdate}, one troy ounce of gold sells for \${ltval:,.2f} '+
#         '(see {\color{orange!40!yellow}\\textbf{---}}), '
#         f'compared to an average of \${prval:,.2f} per ounce during {prdate}. '+
#         'Following the great recession, the monthly average price '+
#         f'of gold reached \${grval:,.2f} per ounce, in {grdate}.')
# print(text)
# write_txt(text_dir / 'gold.txt', text)

### Fed Balance Sheet

In [11]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=38d757c01cb3b550176f371352643679&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data = clean_data / 1_000_000

clean_data.to_csv(data_dir / 'fed_assets.csv', index_label='date', float_format='%g')

In [12]:
ltval = clean_data['RESPPMA_N.WW'].iloc[-1].round(1)
ltdate = dtxt(clean_data.index[-1])['day1']
n = (clean_data.iloc[-1] - clean_data.iloc[-2])['RESPPMA_N.WW'] * 1000
chval = (f'increased by \${abs(n):.0f} billion' if n >= 10 
         else f'decreased by \${abs(n):.0f} billion' 
         if n <= -10 else 'was largely unchanged')

text = ('In response to the collapse of the housing bubble, the Fed purchased '+
        'U.S. Treasury bonds and mortgage-backed securities. Total assets held by '+
        'the Federal Reserve (see {\color{blue!80!black}\\textbf{---}}) '+
        'increased from \$0.9 trillion in August 2008 to '+
        '\$2.2 trillion in November 2008. Additional rounds of asset purchases, '+
        'referred to as quantitative easing, increased the balance sheet to '+
        '\$4.5 trillion by January 2014. As bonds mature they were replaced '+
        'until October 2017, when the Fed allowed the size of its balance '+
        'sheet to normalize. Total assets fell below \$3.8 trillion in August 2019.')
write_txt(text_dir / 'fed_assets1.txt', text)
        
txt2 = ('Balance sheet normalization ended in September 2019 when the Fed increased '+
        'operations in overnight and term repurchase agreement (repo) markets, following '+
        'a sharp increase in rates in these markets. The Fed balance sheet '+
        'increased to \$4.1 trillion by December 2019.')
write_txt(text_dir / 'fed_assets2.txt', txt2)
        
txt3 = ('During the COVID-19 pandemic, the Fed offered lending to businesses and '+
        'currency swaps to major US trading partners, began to purchase commercial bonds, '+
        'and expanded purchases of treasuries and mortgage-backed securities.\n\n '+
        'The Fed balance sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. ')
write_txt(text_dir / 'fed_assets3.txt', txt3)
print(txt3)
        
### Add something about recent pace (e.g. "The total value of Fed assets {chval} from the value one week prior.")

During the COVID-19 pandemic, the Fed offered lending to businesses and currency swaps to major US trading partners, began to purchase commercial bonds, and expanded purchases of treasuries and mortgage-backed securities.

 The Fed balance sheet increased from \$4.2 trillion in February 2020 to \$8.9 trillion, as of the latest data, covering February 2, 2022. 


### Fed Balance Sheet Table

In [13]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=bbb7b50f663fe933c8cce7e3707e3998&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d2 = {'RESPPA_N.WW': '\\textbf{Total} (see {\color{blue!80!black}\\textbf{---}})',
      'RESPPALGUO_N.WW': '\hspace{2mm}U.S. Treasury securities',
      'RESPPALGASMO_N.WW': '\hspace{2mm}Mortgage-backed securities',
      'RESH4SCS_N.WW': '\hspace{2mm}Central bank liquidity swaps',
      'RESPPALGTR_N.WW': '\hspace{2mm}Repurchase agreements',
      'RESPPALD_N.WW': '\hspace{2mm}Loans',
      'RESPPALDJ_N.WW': '\hspace{4mm}Payroll Protection Program',
      'NetPremDisc': '\hspace{2mm}Net unamortized premium',
      'Other': '\hspace{2mm}Other'}

pr_di = ['RESPPALSD_N.WW', 'RESPPALSP_N.WW']

data = clean_data.copy()
data['Other'] = (data['RESPPA_N.WW'] - 
                             data.drop('RESPPA_N.WW', axis=1).sum(axis=1))
data['NetPremDisc'] = data[pr_di].sum(axis=1)
ltval_treas = data['RESPPALGUO_N.WW'].iloc[-1] / 1_000_000
ltval_mbs = data['RESPPALGASMO_N.WW'].iloc[-1] / 1_000_000

data = data.rename(d2, axis=1).drop(pr_di, axis=1)

df = pd.DataFrame()

for i in [-1, -2, -5, -53, -105]:
    df[dtxt(data.index[i])['day2']] = data.iloc[i]
    
(df.loc[list(d2.values()), :].div(1000)
   .applymap('{:,.1f}'.format)
   .to_csv(data_dir / 'fed_bal_sheet.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

text = (f'The Fed currently holds \${ltval_treas:,.1f} trillion in Treasuries '+
        f'and \${ltval_mbs:,.1f} trillion in mortgage-backed securities. ')
write_txt(text_dir / 'fed_assets4.txt', text)
print(text)

The Fed currently holds \$5.7 trillion in Treasuries and \$2.7 trillion in mortgage-backed securities. 


### Exchange rates

In [14]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H10&series=ad1712193ad5bad7b424e3ae5eb101a5&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

for cc in ['EU', 'UK', 'AL', 'NZ']:
    clean_data[f'RXI_N.B.{cc}'] = 1 / clean_data[f'RXI$US_N.B.{cc}'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.dropna(how='all').iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
twidx = clean_data.resample('MS').mean().append(latest)['JRXWTFB_N.B'].dropna()
(clean_data.resample('MS').mean().append(latest)[major]
           .to_csv(data_dir / 'fx1.csv', index_label='date', float_format='%g'))
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', float_format='%g', header=True)

ldate = dtxt(twidx.index[-1])['day1']
lval = twidx.iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx.iloc[-38:].mean()
prev3yr = twidx.iloc[-74:-38].mean()

text = (f'The latest index value, as of {ldate}, is {lval:.1f}, an increase '+
        f'of {totch:.1f} percent since inception in 2006. Over the past three years, '+
        f'the index value has averaged {threeyr:.1f}, compared to an average '+
        f'of {prev3yr:.1f} over the previous three-year period.')
write_txt(text_dir / 'twdbasic.txt', text)
print(text)

The latest index value, as of January 28, 2022, is 116.1, an increase of 16.1 percent since inception in 2006. Over the past three years, the index value has averaged 115.5, compared to an average of 112.7 over the previous three-year period.


In [15]:
cad = clean_data.loc[ldate, 'RXI_N.B.CA']
jpy = clean_data.loc[ldate, 'RXI_N.B.JA'] * 100
eur = clean_data.loc[ldate, 'RXI_N.B.EU']
gbp = clean_data.loc[ldate, 'RXI_N.B.UK']
cadpc = inc_dec_percent(clean_data['RXI_N.B.CA'].pct_change(262).iloc[-1] * 100)
jpypc = inc_dec_percent(clean_data['RXI_N.B.JA'].pct_change(262).iloc[-1] * 100)
gbppc = inc_dec_percent(clean_data['RXI_N.B.UK'].pct_change(262).iloc[-1] * 100)
eurpc = inc_dec_percent(clean_data['RXI_N.B.EU'].pct_change(262).iloc[-1] * 100)

text = (f'As of {ldate}, one US dollar buys approximately: {cad:.2f} Canadian dollars '+
        f'(see {{\color{{green!85!blue}}\\textbf{{---}}}}), {jpy:.0f} Japanese yen '+
        f'(see {{\color{{red}}\\textbf{{---}}}}), {eur:.2f} euros '+
        f'(see {{\color{{cyan!90!white}}\\textbf{{---}}}}), and {gbp:.2f} British pounds '+
         '(see {\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal '+
        f'exchange rate between the US dollar and the Canadian dollar {cadpc}, the USD-JPY rate '+
        f'{jpypc}, the USD-EUR rate {eurpc}, and the USD-GBP rate {gbppc}.')
write_txt(text_dir / 'selcurr_basic.txt', text)
print(text)

As of January 28, 2022, one US dollar buys approximately: 1.28 Canadian dollars (see {\color{green!85!blue}\textbf{---}}), 115 Japanese yen (see {\color{red}\textbf{---}}), 0.90 euros (see {\color{cyan!90!white}\textbf{---}}), and 0.75 British pounds (see {\color{blue!90!cyan}\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar decreased by 0.2 percent, the USD-JPY rate increased by 10.7 percent, the USD-EUR rate increased by 8.4 percent, and the USD-GBP rate increased by 2.0 percent.


### Exchange Rates Table 

In [16]:
fx = {'RXI_N.B.EU': 'EUR',
      'RXI_N.B.UK': 'GBP',
      'RXI_N.B.JA': 'JPY',
      'RXI_N.B.CA': 'CAD',
      'RXI_N.B.MX': 'MXN',
      'RXI_N.B.CH': 'CNY',
      'RXI_N.B.SZ': 'CHF',
      'RXI_N.B.HK': 'HKD',
      'RXI_N.B.IN': 'INR',
      'RXI_N.B.AL': 'AUD',
      'RXI_N.B.NZ': 'NZD',
      'RXI_N.B.BZ': 'BRL',
      'RXI_N.B.KO': 'KRW',
      'RXI_N.B.MA': 'MYR',
      'RXI_N.B.DN': 'DKK',
      'RXI_N.B.NO': 'NOK',
      'RXI_N.B.SD': 'SEK',
      'RXI_N.B.SF': 'ZAR',
      'RXI_N.B.SI': 'SGD',
      'RXI_N.B.TA': 'TWD'}

tbl_data = clean_data[fx.keys()].dropna(how='all')
tbl_data.columns = fx.values()
tbl_data.loc[:,'JPY'] *= 100

table = pd.DataFrame()
table[dtxt(tbl_data.index[-1])['day2']] = tbl_data.iloc[-1]
table['1-month moving average'] = tbl_data.iloc[-22:].mean()
table['1-year moving average'] = tbl_data.iloc[-262:].mean()
dec1 = ['JPY', 'KRW']
table.loc[['JPY', 'KRW'],:] = table.loc[['JPY', 'KRW'],:].applymap("{0:.1f}".format)
dec3 = ['GBP', 'EUR', 'CHF', 'AUD', 'NZD', 'CAD', 'SGD']
table.loc[dec3,:] = table.loc[dec3,:].applymap("{0:.3f}".format)
table.loc[~table.index.isin(dec3+dec1)] = table.loc[~table.index.isin(dec3+dec1)].applymap("{0:.2f}".format)
table['1-month percent change'] = (tbl_data.pct_change(22) * 100).iloc[-1].apply("{0:.1f}".format)
table['1-year percent change'] = (tbl_data.pct_change(262) * 100).iloc[-1].apply("{0:.1f}".format)
table['5-year percent change'] = (tbl_data.pct_change(262*5) * 100).iloc[-1].apply("{0:.1f}".format)

table.index = [f'\includegraphics[width=.03\\textwidth]{{data/flags/{cc}}} \ {cc}' for cc in table.index]

(table.to_csv(data_dir / 'fx_table.tex', sep='&', 
              line_terminator='\\\ ', quotechar=' '))

### Jobless claims

In [17]:
data = fred_df('ICNSA', start='2017') / 1000 
data['1M'] = data['VALUE'].rolling(4).mean()
data.div(1000).to_csv(data_dir / 'icsa.csv', index_label='date', float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest1m = data["1M"].iloc[-1]*1000
prev1m = data["1M"].iloc[-53]*1000

chval = totval - data['VALUE'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = ('The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{report}} '+
        f'{totval:,.0f} actual new claims for unemployment '+
        'insurance (UI) under state programs (see {\color{cyan!80!blue}\\textbf{---}}) '+
        f'during the week ending {datelt}, {change}. Over the past four weeks, '+
        f'new claims have averaged {round(latest1m,-2):,.0f} per week. During the same '+
        f'four-week period last year, there were an average of {round(prev1m,-2):,.0f} '+
        'new claims per week.')
write_txt(text_dir / 'icsa.txt', text)
print(text)
data = fred_df('CCNSA', start='2017') / 1000 
data.div(1000).to_csv(data_dir / 'ccsa.csv', index_label='date', float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
prval = data['VALUE'].iloc[-2]*1000
datelt = dtxt(data.index[-1])['day1']
prevyrval = data['VALUE'].iloc[-53]*1000
prmoval = data['VALUE'].iloc[-4]*1000
datepr = dtxt(data.index[-53])['day1']

chval = totval - prval
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = (f'For the week ending {datelt}, the Department of Labor '+
        '\href{{https://www.dol.gov/ui/data.pdf}}{{reports}} '+
        f'{totval:,.0f} continued claims for unemployment '+
        'insurance (insured unemployed) under state programs '+
        '(see {\color{green!90!blue}\\textbf{---}})'+
        f', {change}. One year prior, during the week of {datepr}, '+
        f'there were an average of {round(prevyrval,-2):,.0f} '+
        'insured unemployed.')
write_txt(text_dir / 'ccsa.txt', text)
print(text)

The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{report}} 257,002 actual new claims for unemployment insurance (UI) under state programs (see {\color{cyan!80!blue}\textbf{---}}) during the week ending January 29, 2022, a one-week decrease of 11,700. Over the past four weeks, new claims have averaged 321,900 per week. During the same four-week period last year, there were an average of 927,600 new claims per week.
For the week ending January 22, 2022, the Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reports}} 2,040,548 continued claims for unemployment insurance (insured unemployed) under state programs (see {\color{green!90!blue}\textbf{---}}), a one-week increase of 39,300. One year prior, during the week of January 23, 2021, there were an average of 5,193,800 insured unemployed.


In [18]:
file = 'https://oui.doleta.gov/unemploy/docs/weekly_pandemic_claims.xlsx'
fed_raw = pd.read_excel(file).iloc[1:]
pua_ic = fed_raw.groupby('Rptdate')['PUA IC'].sum()
pua_ic.index = pd.to_datetime(pua_ic.index)
pua_ic.index.name = 'date'
pua_ic.name = 'pua_ic'
cc_raw = fed_raw.dropna(subset=['PUA CC'])
pua_cc = cc_raw.groupby('Rptdate')['PUA CC'].sum()
peuc_cc = cc_raw.groupby('Rptdate')['PEUC CC'].sum()
fed_cc = pua_cc + peuc_cc
fed_cc.index = pd.to_datetime(fed_cc.index)
fed_cc.index.name = 'date'
fed_cc.name = 'fed_cc'

d1 = pd.DataFrame([pua_ic, fed_cc]).T / 1000
d1.div(1000).to_csv(data_dir / 'fed_uic.csv', index_label='date', float_format='%g')

ltdate = dtxt(d1.index[-1])['day1']
prdatecc = dtxt(d1.dropna().index[-4])['day1']
ltdatecc = dtxt(d1.dropna().index[-1])['day1']
pua_ic_lt = f"{round(d1['pua_ic'].iloc[-1] * 1000, -2):,.0f}"
pua_ic_pr = f"{round(d1['pua_ic'].iloc[-2] * 1000, -2):,.0f}"
pua_ic_1m = f"{round(d1['pua_ic'].iloc[-4:].mean() * 1000, -2):,.0f}"
fed_cc_lt = d1.dropna()['fed_cc'].iloc[-1] * 1_000
fed_cc_mo = d1.dropna()['fed_cc'].iloc[-4] * 1_000
fed_cc_ltt = f"{fed_cc_lt:,.0f}"
tot_cc = fed_cc_lt + totval
tot_lt = f"{tot_cc / 1_000_000:,.1f} million"
tot_cc_pr = fed_cc_mo + prmoval
tot_pr = f"{tot_cc_pr / 1_000_000:,.1f} million"

text = (f'Over the week ending {ltdate}, there were {pua_ic_lt} '+
        'initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} '+
        'under the Pandemic Unemployment Assistance '+
        '(PUA) program (see {\color{blue!50!purple!80!black}\\textbf{---}}), '+
        f'compared to {pua_ic_pr} during the prior week, '+
        f'and an average of {pua_ic_1m} initial claims per week over the '+
        'past four weeks. \n\n'+
        f'Federal program continuing claims total {fed_cc_ltt} '+
        f'in {ltdatecc} '+
        '(see {\color{green!50!blue}\\textbf{---}}). These include both ' +
        'claims under the PUA program and claims under the Pandemic Emergency '+
        'Unemployment Compensation (PEUC) program. Combining federal program '+
        f'claims with state program claims indicates there are a total '+
        f'of {tot_lt} insured unemployed persons during the week ending {ltdatecc}, '+
        f'compared to {tot_pr} one month prior, during the week ending {prdatecc}. ')
write_txt(text_dir / 'fed_uic.txt', text)
print(text)

Over the week ending January 29, 2022, there were 600 initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} under the Pandemic Unemployment Assistance (PUA) program (see {\color{blue!50!purple!80!black}\textbf{---}}), compared to 500 during the prior week, and an average of 700 initial claims per week over the past four weeks. 

Federal program continuing claims total 128,179 in January 22, 2022 (see {\color{green!50!blue}\textbf{---}}). These include both claims under the PUA program and claims under the Pandemic Emergency Unemployment Compensation (PEUC) program. Combining federal program claims with state program claims indicates there are a total of 2.2 million insured unemployed persons during the week ending January 22, 2022, compared to 2.2 million one month prior, during the week ending January 1, 2022. 


### VIX (SP500 volatility)

In [19]:
#prev = pd.read_excel(data_dir/ 'vixarchive.xls', skiprows=1, index_col='Date')
VIX = 'https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv'
curr = pd.read_csv(VIX, index_col='DATE', parse_dates=True)
df = (curr['CLOSE'].resample('MS').mean()
      .append(curr['CLOSE'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')

node = end_node(df, 'magenta', date='d', offset=0.35, full_year=True)
write_txt(text_dir / 'vix_node.txt', node)

ldate = dtxt(df.index[-1])['day1']
vallt = df.iloc[-1]
val3y = df.iloc[-37:].mean()

compare = compare_text(vallt, val3y, [3, 12, 30])

one_wk = (value_text(curr.CLOSE.diff(5).iloc[-1], style='increase_by', ptype='pp')
          .replace('percentage ', ''))

text = ('This volatility measure, the VIX index (see {\color{magenta}\\textbf{---}}), '+
        f'was {vallt:.1f} on {ldate}, '+
        f'{compare} the average index value of {val3y:.1f} over the '+
        f'past three years. The VIX {one_wk} over the past week.')
write_txt(text_dir / 'vixbasic.txt', text)
print(text)

This volatility measure, the VIX index (see {\color{magenta}\textbf{---}}), was 24.4 on February 3, 2022, in line with the average index value of 21.8 over the past three years. The VIX decreased by 6.1 points over the past week.


### Oil prices (WTI)

In [20]:
df = fred_df('DCOILWTICO')
latest = df.iloc[-1]
ma = df.resample('MS').mean()
p = ma.append(latest)['VALUE']
(p.to_csv(data_dir / 'wti.csv', index_label='date', header=True))

oneyr = p.pct_change(13).iloc[-1] * 100
twoyr = p.pct_change(25).iloc[-1] * 100

oyt = value_text(oneyr, casual=True)
tyt = value_text(twoyr)

node = end_node(p, 'red!80!purple', dollar=True, digits=2, date='d', 
                offset=0.35, full_year=True)
write_txt(text_dir / 'oil_node.txt', node)
    
ltch = p.loc['2008-06-01'] - p.iloc[-1]
ldate = dtxt(p.index[-1])['day1']
text = (f'As of {ldate}, a barrel of west Texas '+
        f'intermediate (WTI) \\textbf{{crude oil}} sells for \${p.iloc[-1]:.2f} '+
        '(see {\color{red!80!purple}\\textbf{---}}). Over the past year, '+
        f'this measure of oil prices {oyt}. Over the past two years, the '+
        f'price {tyt}. Notably, the WTI price is currently \${ltch:.0f} per barrel '+
        'below its peak price of \$134 per barrel in June 2008 (monthly average).')
write_txt(text_dir / 'wti.txt', text)
print(text)

As of January 31, 2022, a barrel of west Texas intermediate (WTI) \textbf{crude oil} sells for \$89.16 (see {\color{red!80!purple}\textbf{---}}). Over the past year, this measure of oil prices grew 71.4 percent. Over the past two years, the price increased 55.0 percent. Notably, the WTI price is currently \$45 per barrel below its peak price of \$134 per barrel in June 2008 (monthly average).


### Inflation Expectations

In [21]:
data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', header=True)
data2 = fred_df('T5YIFR').loc['2015':,'VALUE']
df = pd.DataFrame({'5_year_breakeven': data1, '5_year_5_year_forward': data2})
df.to_csv(data_dir / 'infbreak_comb.csv', index_label='date')

node = end_node(data1, 'blue!70!black', percent=True)
write_txt(text_dir / 'infbreak_node.txt', node)

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.dropna().index[-252])['day1']
pvalm = data1.dropna().iloc[-252]
p5valm = data1.dropna().iloc[-(252*5)]

text = (f'As of {ldatem}, markets expect an average inflation rate of {lvalm:.1f} ' + 
         'percent over the next five years (see {\color{blue!70!black}\\textbf{---}}), '
         +f'compared to an expected rate of {pvalm:.1f} percent on '+
         f'{pdatem}. Markets had expected inflation to average {p5valm:.1f} percent '+
         f'per year over the past five years, five years ago.')
write_txt(text_dir / 'inf_exp_mkts.txt', text)
print(text)

p55val = data2.iloc[-1]
if data2.iloc[-1] + 0.1 > data1.iloc[-1]:
    compare = 'fall below '
elif data2.iloc[-1] - 0.1 < data1.iloc[-1]:
    compare = 'exceed '
else:
    compare = 'maintain the same rate as '
text = (f'Over this five-year period, markets suggest {p55val:.1f} percent inflation '+
        f'per year. Inflation rates in the near-term are therefore expected to {compare}'+
        'inflation rates in the longer-term')
write_txt(text_dir / 'inf_exp_mkts_55.txt', text)

As of February 4, 2022, markets expect an average inflation rate of 2.8 percent over the next five years (see {\color{blue!70!black}\textbf{---}}), compared to an expected rate of 2.3 percent on February 5, 2021. Markets had expected inflation to average 2.0 percent per year over the past five years, five years ago.
